# Accessing Census data with Python

This notebook walks you through the process of getting data using Datamade's [census library for Python](https://github.com/datamade/census). At the same time we will introduce you to the concept of uncertainty in sample data and how to work with it.

## Importing the tools we need to work with data
First we import the libraries we are going to use to get data and analyze it. We are using three main libraries:
- **[Pandas:](https://pandas.pydata.org/)** The toolkit we use to conduct data analysis
- **[Census:](https://github.com/datamade/census)** This library makes it easier to access Census data through the API.
- **[Altair:](https://altair-viz.github.io/)** This library is used to create data visualizations using data in Pandas.

We also need to import the API key so we can use the Census API.

In [32]:
import altair as alt
from census import Census
import pandas as pd

from census_api_key import KEY

## Starting the Census client

Next we need to initialize the client to download data. The client only needs one bit of information from you in order to work - your Census API key.

In [33]:
client = Census(KEY)

## Using the client to search for data

The client has functionality built in to list all the tables associated with a given dataset. In this example, we want to look at the median household income in Santa Clara County using 1-year estimates from the American Community Survey.

You can list all the tables using the following syntax:

In [ ]:
tables = client.acs1.tables()
# show the number of tables
len(tables)

In [ ]:
# showing the first 5 tables
tables[:5]

The result is a list of tables, each one with it's own dictionary:
- **name:** This is the table ID, an important bit of information needed to download data through the API.
- **description:** A human-readable description of the table.
- **variables:** A url pointing to the variable definitions for the table.
- **universe:** The universe the ACS estimates are based on.

We can use **name** to find our data easily. The first letter of the table ID describes the type of table.
We oftenly use tables starting with the letter `B` - basic, detailed tables. Next we need to find the tables associated with income. The first two digits after the letter in the table ID identify the subject. For income those numbers are `19`. A full listing of subjects and their numbers can be found [here](https://www.census.gov/programs-surveys/acs/data/data-tables/table-ids-explained.html). A [spreadsheet listing all tables](https://www2.census.gov/programs-surveys/acs/tech_docs/table_shells/table_lists/2022_DataProductList.xlsx) is also available.

For now let's find all tables related to income:

In [ ]:
income_tables = []

for table in client.acs1.tables():
    if table['name'].startswith('B19'):
        income_tables.append(table)

len(income_tables)

There are more than 100 tables related to income, still too many to list here. Let's refine our search a bit more, but let's print out some information instead of storing it in a list:

In [ ]:
for table in client.acs1.tables():
    if table['name'].startswith('B19') and 'MEDIAN HOUSEHOLD' in table['description'].upper():
        print(table['name'], table['description'])

Much better. It may be hard to see, but the table we want is in there with a table ID of **B19013**.
Now we want to look at the variables within the table, using the url that is listed in the data. However, the url points to a JSON object containing the variables. We want it in HTML for easier reading. This can be done pretty easily in Python:

In [ ]:
for table in client.acs1.tables():
    if table['name'] == 'B19013':
        url = table['variables']
        print(url.replace('json', 'html'))

Click on the link to view the variables. You will notice the table IDs now have some extra characters at the end. They all now have `_001` which identifies the column. The letters at the end define the type of measure. We are only interested in those that end in `E` for estimate and `M` for margin of error.
## Downloading data
So now let's define a few things to make it easier for us to query the API and download the relevant data:

In [40]:
state_fips = '06' # California state FIPS code
county_fips = '085' # FIPS code for Santa Clara County

We also need to specify what fields we want to download from the table we've identified. I like to do this in a dictionary where the keys are the variable IDs and the values are human-readable names of the variables. I do this so I can easily see what these are in my code and to make renaming columns easier later on.

In [41]:
fields = {
    'B19013_001E': 'median_hh_income',
    'B19013_001M': 'margin_of_error',
}
# I also create a list of the variable IDs to give to the Census client
field_codes = list(fields.keys())

We want to use the client's `state_county` function to download the appropriate data. It takes three arguments - the fields we want to download, the state FIPS code and the county FIPS code.

In [ ]:
county_data = client.acs1.state_county(field_codes, state_fips, county_fips)
county_data

We now have a list containing a dictionary with our estimates. Let's add a bit more metadata to this. This is not necessary, we are doing it so we can do some analysis later on.

In [ ]:
for row in county_data:
    row['dataset'] = '1-year estimate'
    row['geo_name'] = 'Santa Clara County'
county_data

Next, let's convert this to a dataframe and rename the columns so they are easier to read.

In [ ]:
data = pd.DataFrame(county_data)
#inplace modifies the dataframe in place, rather than returning a new dataframe
data.rename(columns=fields, inplace=True) 

data

## Margin of error

This section of the notebook replicates the information in the [README](./README.md#Margins-of-error).

Margins of error tell us the reliability of data collected from surveys where sampling is used. The smaller the sample size, the larger the margin of error.

Let's look at our median household income data again.

In [ ]:
data

We can say the true median household income is within \\$2,669 of the published estimate of $141,562. 

If we subtract the margin of error from the published estimate, we get the lower bound of what is known as the confidence interval.

In [ ]:
data['lower_bound'] = data['median_hh_income'] - data['margin_of_error']
data['lower_bound']

Adding the margin of error to the estimate gives us the confidence interval's upper bound.

In [ ]:
data['upper_bound'] = data['median_hh_income'] + data['margin_of_error']
data['upper_bound']

The confidence interval is the range in which we can say with some degree of confidence the estimate's true value exists.

In [ ]:
data[['median_hh_income', 'margin_of_error','lower_bound','upper_bound']]

In this example, we can say with some degree of confidence that the true median household income is somewhere between \$138,893 and \\$144,231. Just how much confidence? The Census Bureau publishes their estimates at the 90 percent confidence level. So we can say the following:
> We are 90 percent confident the median household income for Santa Clara County is between \\$138,893 and \$144,231.

**IMPORTANT:** You should always keep an eye on the margin of error. Look for cases where the lower bound of the confidence interval is less than zero or no longer makes sense.

We can reduce the margin of error and make our estimates more reliable by increasing the sample size. There are a couple of ways to do that with Census data.

### Use a larger geography - adding state data

The easiest way to increase sample size is to increase the size of the geography. In this case, let's add to the table the median household income for the state of California.

The procedure to get state-level data is mostly the same as above, but we will use the Census client's `state` function rather than the `state_county` function.

In [ ]:
state_data = client.acs1.state(field_codes, state_fips)
state_data

Let's add in the extra metadata:

In [ ]:
for row in state_data:
    row['dataset'] = '1-year estimate'
    row['geo_name'] = 'State of California'
state_data

Combine the state data with the county data from above:

In [ ]:
combined = county_data + state_data
combined

In [ ]:
data = pd.DataFrame(combined)
data.rename(columns=fields, inplace=True)
data

Next let's calculate the bounds of our confidence intervals:

In [ ]:
data['lower_bound'] = data['median_hh_income'] - data['margin_of_error']
data['upper_bound'] = data['median_hh_income'] + data['margin_of_error']

data

The margin of error is much lower at the state level, but not without cost. The state estimate also includes other counties, not just Santa Clara County. But the state estimate doesn't accurately reflect the county estimate. This can be problematic if the scope of the analysis needs to be at the county level.

If the analysis focuses on small geographies such as census tracts, consider switching to a county-level analysis to increase the reliability of the data.

### Use 5-year estimates
Another way of increasing sample size is to use 5-year estimates. These estimates are based on survey responses collected over a 5-year period, so the sample size is significantly larger.

Begin by getting the county-level 5-year estimates. We are going to use the `state_county` function just as we did above, but this time we will switch to the `acs5` client to get the correct data.

In [ ]:
county_data_5yr = client.acs5.state_county(field_codes, state_fips, county_fips)
county_data_5yr

Again we will add some metadata, making sure to specify `5-year estimate` as the `dataset`.

In [ ]:
for row in county_data_5yr:
    row['dataset'] = '5-year estimate'
    row['geo_name'] = 'Santa Clara County'
county_data_5yr

Combine our data, load it into a dataframe, rename the columns and calculate the confidence interval:

In [ ]:
combined = county_data + county_data_5yr
data = pd.DataFrame(combined)
data.rename(columns=fields, inplace=True)

data['lower_bound'] = data['median_hh_income'] - data['margin_of_error']
data['upper_bound'] = data['median_hh_income'] + data['margin_of_error']

data

### Visualizing confident intervals

We can use Altair to visualize confidence intervals. Visualization is beyond the scope of this notebook, so we won't go too deep into how to create charts.

#### Create the estimates chart

We will use `mark_point` to create a point chart and set the horizontal (X) axis to the median household income. The vertical (Y) axis, will be the dataset.

In [ ]:
estimates = alt.Chart(
    data
).mark_point(
    filled=True,
    color='black'
).encode(
    alt.X('median_hh_income').scale(zero=False),
    y='dataset'
)
estimates

To chart confidence intervales, we will use `mark_errorbar`. Note we specify two encodings for the X axis - one for the upper bound of the confidence interval and a second for the lower bound.

In [ ]:
confidence_intervals = alt.Chart(
    data
).mark_errorbar(
    color='red'
).encode(
    alt.X('upper_bound').scale(zero=False),
    alt.X2('lower_bound'),
    y='dataset'
)
confidence_intervals

Next we can combine the two charts to create a composite. We also can adjust some properties of the chart such as height.

In [ ]:
(estimates + confidence_intervals).properties(height=150)

We can easily see the confidence interval is much smaller and the estimates look closer to each other. Most times we opt to use 5-year estimates.

**IMPORTANT:**

- Remember year-to-year analysis should not be done because 5-year estimates containing data from overlapping years.
- Estimates at the tract level will still have high margins of error that need to be carefully considered when planning a methodology.

### Avoid segmenting the population
If possible, avoid segmenting the population into demographic subgroups. This can be difficult because we often want to look at things and consider other factors such as age, sex, race and/or ethnicity. 

However whenever we do so, the margin of error will most likely increase because the sample size is reduced.

Let's download 5-year estimates of median household income for multi-racial households:

In [ ]:
multiracial_fields = {
    'B19013G_001E': 'median_hh_income', #notice the slight change to table IDs
    'B19013G_001M': 'margin_of_error',
}
multiracial_codes = list(multiracial_fields.keys())
multiracial_data = client.acs5.state_county(multiracial_codes, state_fips, county_fips)
multiracial_data

Add metadata

In [ ]:
for row in multiracial_data:
    row['dataset'] = '5-year estimate'
    row['geo_name'] = 'Santa Clara County'
multiracial_data

We are going to take a slightly different approach when combining our data. Begin by prepping two dataframes, one for multiracial households and another for all households. We do this because the table IDs are slightly different between the two estimates. We are also going to add a column distinguishing the two different types of households - `all` and `multiracial`.

In [ ]:
multiracial = pd.DataFrame(
    multiracial_data
).rename(
    columns=multiracial_fields
) # Notice we don't use inplace=True because we want something returned to the variable
multiracial['household_type'] = 'multiracial'
multiracial

In [ ]:
all_hh = pd.DataFrame(
    county_data_5yr
).rename(
    columns=fields
)
all_hh['household_type'] = 'all'
all_hh

Now we can combine the two sets of estimates using `pd.concat`

In [ ]:
data = pd.concat([all_hh, multiracial])
data

Ok great, now let's calculate the confidence intervals and visualize the results.

In [ ]:
data['lower_bound'] = data['median_hh_income'] - data['margin_of_error']
data['upper_bound'] = data['median_hh_income'] + data['margin_of_error']

estimates = alt.Chart(
    data
).mark_point(
    filled=True,
    color='black'
).encode(
    alt.X('median_hh_income').scale(zero=False),
    y='household_type' # swith Y axis to `household_type`
)

confidence_intervals = alt.Chart(
    data
).mark_errorbar(
    color='red'
).encode(
    alt.X('upper_bound').scale(zero=False),
    alt.X2('lower_bound'),
    y='household_type'
)

(estimates + confidence_intervals).properties(height=150)

We can easily see the confidence interval for multiracial households is much larger than that for all households. However, since they do not overlap, we can safely make the following statement:

> Multi-racial households earn less than the county's median income.

## Downloading multiple geographic areas

Often we don't want to look at a single county or state. We can easily download all states in the nation, or counties in a state using a property of the Python Census library - `Census.ALL`. This is exactly the same as using `'*'` which is traditionally viewed as a character to represent wildcards.

### Download all states

In [ ]:
raw = client.acs5.state(field_codes, Census.ALL)
len(raw)

In [ ]:
data = pd.DataFrame(raw).rename(columns=fields)
data.head()

We don't have state names with this data, only FIPS codes. We can easily use the crosswalk the Census Bureau provides. The crosswalk is a pipe-delimited text file we can load directly into Pandas without downloading.

In [ ]:
state_fips_url = 'https://www2.census.gov/geo/docs/reference/codes2020/national_state2020.txt'
# Notice we specified `|` for the delimiter
# We are also going to set the data type of all columns as a string 
# to keep leading zeroes in identifiers
state_codes = pd.read_csv(state_fips_url, delimiter='|', dtype=str) 
state_codes.head()

Let's merge these together.

In [ ]:
merged = data.merge(
    state_codes,
    how='left',
    left_on='state',
    right_on='STATEFP'
)
merged.head()

Great we can now write out our results to a file for later use.

In [70]:
merged.to_csv('./median_hh_income_states.csv', index=False)

### Download multiple counties

The process is very similar to above, only we use the client's `state_county` method to get county-level data.

#### For a single state

In [ ]:
raw = client.acs5.state_county(field_codes, state_fips, Census.ALL)
len(raw)

#### For all states

In [ ]:
raw = client.acs5.state_county(field_codes, Census.ALL, Census.ALL)
len(raw)

In [ ]:
data = pd.DataFrame(raw).rename(columns=fields)
data

#### Add county names

We can also use FIPS codes to add county and state names. We will use a different dataset this time that includes county FIPS codes.

In [ ]:
county_fips_url = 'https://www2.census.gov/geo/docs/reference/codes2020/national_county2020.txt'
county_codes = pd.read_csv(county_fips_url, delimiter='|', dtype=str)
county_codes

County FIPS codes are unique within a state, but not nationally. So we need to include both state and county FIPS codes when joining the data.

In [ ]:
data['geoid'] = data['state'] + data['county']
data.head()

In [ ]:
county_codes['geoid'] = county_codes['STATEFP'] + county_codes['COUNTYFP']
county_codes

Now we can join the two dataframes as we did above using the `geoid` columns we just created.

In [ ]:
merged = data.merge(
    county_codes,
    how='left',
    on='geoid'
)
merged.head()

And once again write out our data:

In [78]:
merged.to_csv('./median_hh_income_counties.csv', index=False)